# MoM implementation in Julia
# Initiated by Lisette 
# Editted by Domenico in Attempt to Render Implementation Type-Stable 

## Import Packages 

In [1]:
using LinearAlgebra
using StructArrays 
using Statistics
using StaticArrays
#using PhysicalConstants.CODATA2018
using BenchmarkTools

In [2]:
# a point in 3D is a tuple of 3 coordinates 
# we here introduce static vectors that appear to be vital to reduce the number of allocations 
const Point3D = SVector{3,Float64};

## Section 1: Introduction 
1. method of moments as in Morandi;
2. magnetization vector assumed constant per element - shape functions are element-wise constant;
3. collaction or point matching - test function are Dirac delta distributions; 
4. more later; 

## Section 2: Space for Experimentation 

In [3]:
# generate list of vertices 
# here we consider the list of vertices only 
# this function is intended for testing  
function gen_multi_box_vertices(length::Float64, width::Float64, height::Float64, 
                                num_boxes_x::Int64, num_boxes_y::Int64, num_boxes_z::Int64)

    # define a setv of points 
    x0=0.; y0=0.; z0=0.; x1=1.; y1=1.;z1=1.;
    v1 = Point3D(x0,y0,z0) 
    v2 = Point3D(x1,y0,z0)
    v3 = Point3D(x1,y1,z0) 
    v4 = Point3D(x0,y1,z0)
    v5 = Point3D(x0,y0,z1) 
    v6 = Point3D(x1,y0,z1)
    v7 = Point3D(x1,y1,z1) 
    v8 = Point3D(x0,y1,z1)
    # define the total number of boxes  
    num_boxes = num_boxes_x * num_boxes_y * num_boxes_z 
    # define the vertex type - this type contains the (x,y,z)-coordinates of the 8 vertices of block 
    vertextype   = typeof(@SVector [v1,v2,v3,v4,v5,v6,v7,v8])
    # define num_boxes sized vectors of 8-sized SVectors of 3-sized SVectors 
    vertices_list = Vector{vertextype}(undef,num_boxes)   
    count_boxes = 1; 
    for i in 1:num_boxes_x
        for j in 1:num_boxes_y
            for k in 1:num_boxes_z
                x_start = (i - 1) * length
                x_end = i * length
                y_start = (j - 1) * width
                y_end = j * width
                z_start = (k - 1) * height
                z_end = k * height

                vertices = vertextype(
                    Point3D(x_start, y_start, z_start),
                    Point3D(x_end, y_start, z_start),
                    Point3D(x_end, y_end, z_start),
                    Point3D(x_start, y_end, z_start),
                    Point3D(x_start, y_start, z_end),
                    Point3D(x_end, y_start, z_end),
                    Point3D(x_end, y_end, z_end),
                    Point3D(x_start, y_end, z_end)
                )   

                vertices_list[count_boxes] = vertices
                            
                count_boxes += 1 
                                
            end
        end
    end

    return vertices_list
end

gen_multi_box_vertices (generic function with 1 method)

In [4]:
length_box = 1.0; width = 1.0; height = 1.0
num_boxes_x = 10; num_boxes_y = 1; num_boxes_z = 1
vertices_list = gen_multi_box_vertices(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z)

length_box = 1.0; width = 1.0; height = 1.0
num_boxes_x = 10; num_boxes_y = 1; num_boxes_z = 1
@btime gen_multi_box_vertices(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z)

length_box = 1.0; width = 1.0; height = 1.0
num_boxes_x = 100; num_boxes_y = 1; num_boxes_z = 1
@btime gen_multi_box_vertices(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z); 

  63.744 ns (1 allocation: 1.98 KiB)
  432.607 ns (2 allocations: 18.80 KiB)


In [5]:
# generate list of vertices 
# here we consider the list of vertices only 
# this function is intended for testing  
function gen_multi_box(length::Float64, width::Float64, height::Float64, 
                       num_boxes_x::Int64, num_boxes_y::Int64, num_boxes_z::Int64)

    # define a setv of points 
    x0=0.; y0=0.; z0=0.; x1=1.; y1=1.;z1=1.;
    v1 = Point3D(x0,y0,z0) 
    v2 = Point3D(x1,y0,z0)
    v3 = Point3D(x1,y1,z0) 
    v4 = Point3D(x0,y1,z0)
    v5 = Point3D(x0,y0,z1) 
    v6 = Point3D(x1,y0,z1)
    v7 = Point3D(x1,y1,z1) 
    v8 = Point3D(x0,y1,z1)
    # define the total number of boxes  
    num_boxes = num_boxes_x * num_boxes_y * num_boxes_z 
    # define the vertex type - this type contains the (x,y,z)-coordinates of the 8 vertices of block 
    vertextype   = typeof(@SVector [v1,v2,v3,v4,v5,v6,v7,v8])
    triangletype = typeof(@SVector [v1,v2,v3])
    t1 = triangletype(v1,v2,v3);  t2  = triangletype(v1,v3,v4)  
    t3 = triangletype(v5,v6,v7);  t4  = triangletype(v5,v7,v8)  
    t5 = triangletype(v1,v2,v6);  t6  = triangletype(v1,v6,v5)  
    t7 = triangletype(v3,v4,v8);  t8  = triangletype(v3,v8,v7)  
    t9 = triangletype(v2,v3,v7);  t10 = triangletype(v2,v7,v6)  
    t11 = triangletype(v1,v4,v8); t12 = triangletype(v1,v8,v5)  
    facetype     = typeof(@SVector [t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12])
    # define num_boxes sized vectors of 8-sized SVectors of 3-sized SVectors 
    vertices_list = Vector{vertextype}(undef,num_boxes)   
    faces_list    = Vector{facetype}(undef,num_boxes)   
    count_boxes = 1; 
    for i in 1:num_boxes_x
        for j in 1:num_boxes_y
            for k in 1:num_boxes_z
                x_start = (i - 1) * length
                x_end = i * length
                y_start = (j - 1) * width
                y_end = j * width
                z_start = (k - 1) * height
                z_end = k * height

                vertices = vertextype(
                    Point3D(x_start, y_start, z_start),
                    Point3D(x_end, y_start, z_start),
                    Point3D(x_end, y_end, z_start),
                    Point3D(x_start, y_end, z_start),
                    Point3D(x_start, y_start, z_end),
                    Point3D(x_end, y_start, z_end),
                    Point3D(x_end, y_end, z_end),
                    Point3D(x_start, y_end, z_end)
                )   

                vertices_list[count_boxes] = vertices
                
                faces = facetype(
                    triangletype(vertices[1], vertices[2], vertices[3]),
                    triangletype(vertices[1], vertices[3], vertices[4]),

                    triangletype(vertices[5], vertices[6], vertices[7]),
                    triangletype(vertices[5], vertices[7], vertices[8]),

                    triangletype(vertices[1], vertices[2], vertices[6]),
                    triangletype(vertices[1], vertices[6], vertices[5]),

                    triangletype(vertices[3], vertices[4], vertices[8]),
                    triangletype(vertices[3], vertices[8], vertices[7]),

                    triangletype(vertices[2], vertices[3], vertices[7]),
                    triangletype(vertices[2], vertices[7], vertices[6]),

                    triangletype(vertices[1], vertices[4], vertices[8]),
                    triangletype(vertices[1], vertices[8], vertices[5]) 
                ) 

                faces_list[count_boxes] = faces 
                
                count_boxes += 1 
                                
            end
        end
    end

    return vertices_list, faces_list 
end

gen_multi_box (generic function with 1 method)

In [6]:
length_box = 1.0; width = 1.0; height = 1.0
num_boxes_x = 10; num_boxes_y = 1; num_boxes_z = 1
@btime gen_multi_box(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z)

length_box = 1.0; width = 1.0; height = 1.0
num_boxes_x = 100; num_boxes_y = 1; num_boxes_z = 1
@btime gen_multi_box(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z); 

length_box = 1.0; width = 1.0; height = 1.0
num_boxes_x = 1000; num_boxes_y = 1; num_boxes_z = 1
@btime gen_multi_box(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z); 

  278.257 ns (2 allocations: 10.55 KiB)
  2.093 μs (4 allocations: 103.22 KiB)
  19.000 μs (4 allocations: 1.01 MiB)


## Section 3: Mesh Generation 
We first define 3 user defined types, namely Point3D, Vertices and Faces  

## Section 4: Utility Functions 

In [ ]:
function calculate_normal(face_vertices)
    u = face_vertices[2] - face_vertices[1]
    v = face_vertices[3] - face_vertices[1]

    normal_vector = cross(u, v)
    normal_unit_vector = normal_vector / norm(normal_vector)

    return normal_unit_vector
end

function calculate_normal!(result,face_vertices)
    u = face_vertices[2] - face_vertices[1]
    v = face_vertices[3] - face_vertices[1]

    result = myCross!(result,u, v)
    result = result / norm(result)

    return result 
end

function find_center_r_k(vertices)
    return sum(vertices) / length(vertices)
end

function r_norm(r_1, r_2, epsilon)
    return sqrt(norm(r_1 - r_2)^2 + epsilon^2)
end

function find_w_e(r_1, r_2, r_k, epsilon)
    return log((r_norm(r_2, r_k, epsilon) + r_norm(r_1, r_k, epsilon) + r_norm(r_2, r_1, epsilon)) / (r_norm(r_2, r_k, epsilon) + r_norm(r_1, r_k, epsilon) - r_norm(r_2, r_1, epsilon)))
end

function solid_angle_triangular_plane(r_1, r_2, r_3, r_k, epsilon)
    D_r = r_norm(r_1, r_k, epsilon) * r_norm(r_2, r_k, epsilon) * r_norm(r_3, r_k, epsilon) + r_norm(r_1, r_k, epsilon) * dot((r_2 - r_k), (r_3 - r_k)) + r_norm(r_2, r_k, epsilon) * dot((r_1 - r_k), (r_3 - r_k)) + r_norm(r_3, r_k, epsilon) * dot((r_1 - r_k), (r_2 - r_k))
    x1 = dot((r_1 - r_k), SVector{3}(cross(SVector{3}((r_2 - r_k)), SVector{3}(((r_3 - r_k))))))
    return 2 * atan(x1, D_r)
end

function myCross!(result,a,b)
# computes outer product of a and b and stores results in w
# assumption is that memory for w has been allocated outside this function
# not sure whether this is necessary at all     
    result[1] = a[2]*b[3] - a[3]*b[2]
    result[2] = -a[1]*b[3] + a[3]*b[1]
    result[3] = a[1]*b[3] - a[3]*b[1]
    return result
end 

## Section 4: Code by Lisette to Compute the Matrix 

In [ ]:
function orig_build_C_matrix(vertices_list, faces_list)
    N = length(vertices_list)
    eps = 0.001
    I_3 = Matrix{Float64}(I, 3, 3)
    C = zeros(3 * N, 3 * N)


    for (ele, vertex_set) in enumerate(vertices_list)
        r_k = find_center_r_k(vertex_set)

        for (source, face_set) in enumerate(faces_list)
            
            #C_subblock = SMatrix{3, 3}(zeros(3, 3))
            C_subblock = zeros(3, 3)
            for (S_f, face) in enumerate(face_set)
                
                # Calculate the normal vector on the plane
                n_f = calculate_normal(face)

                # Calculate center of the face
                r_k_face = find_center_r_k(vertices_list[source])

                # Calculate vector from vertex to center
                centroid_face = mean(face, dims=1)
                
                if dot(n_f, r_k_face - centroid_face[1]) > 0
                    n_f = -n_f
                    face = [face[1], face[3], face[2]]
                end

                r_1 = face[1]
                r_2 = face[2]
                r_3 = face[3]

                solid_angle = solid_angle_triangular_plane(r_1, r_2, r_3, r_k, eps)

                t_e_w_e = zeros(Float64,3,)
                for (v, vertex) in enumerate(face)
                    v_next = mod(v, 3) + 1
                    current_edge = face[v_next] - face[v]

                    t_e = current_edge / norm(current_edge)
                    w_e = find_w_e(face[v], face[v_next], r_k, eps)
                    t_e_w_e = t_e_w_e + w_e* t_e
                end

                gradient_W = cross(n_f, cross(t_e_w_e, n_f)) + n_f * solid_angle
                
                for i in 1:size(I_3, 1)
                    u_i = I_3[:, i]
                    u_i_transposed = reshape(u_i, 1, :)
                    #C_subblock -= SMatrix{3,3}((1.25663706212e-6/ (4 * pi)) * cross(cross(u_i, n_f), gradient_W) * u_i_transposed)
                    C_subblock -= ((1.25663706212e-6/ (4 * pi)) * cross(cross(u_i, n_f), gradient_W) * u_i_transposed)
                    
                end
            end

            C[3 * (ele - 1) + 1:3 * ele, 3 * (source - 1) + 1:3 * source] = C_subblock
        end
    end

    return C
end

In [ ]:
length_box = 1.0
width = 1.0
height = 1.0

num_boxes_x = 2; num_boxes_y = 1; num_boxes_z = 1
vertices_list, faces_list = generate_multi_box(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z)
@btime C = orig_build_C_matrix(vertices_list, faces_list)

## Section 6: Sandbox for Domenico to Play in 
The function blank_C_matrix() allocates too much. 

In [ ]:
length_box = 1.
width = 1.
height = 1.

num_boxes_x = 2
num_boxes_y = 1
num_boxes_z = 1

vertices_list, faces_list = generate_multi_box(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z);

In [ ]:
function blank_C_matrix(vertices_list, faces_list)
    N = length(vertices_list)
    eps = 0.001
    I_3 = Matrix{Float64}(I, 3, 3)
    C = zeros(3 * N, 3 * N)
    # added by domenico 
    C_subblock = zeros(Float64,3,3) 
    nf = zeros(3)
    w1 = zeros(3) # first temporary vector of size 3 
    w2 = zeros(3) # first temporary vector of size 3 

    for (ele, vertex_set) in enumerate(vertices_list)

        r_k = find_center_r_k(vertex_set)

        for (source, face_set) in enumerate(faces_list)
                        
            for (S_f, face) in enumerate(face_set)
                
                # Calculate the normal vector on the plane
                n_f = calculate_normal!(w1,face)

                for i in 1:3
                    for j in 1:3
                        C_subblock[i,j] -= 1.0*(i+j)
                    end 
                end
            end

            C[3 * (ele - 1) + 1:3 * ele, 3 * (source - 1) + 1:3 * source] = C_subblock
        end
    end

    return C
end

In [ ]:
length_box = 1.0
width = 1.0
height = 1.0

num_boxes_x = 2; num_boxes_y = 1; num_boxes_z = 1
vertices_list, faces_list = generate_multi_box(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z)
@btime C = blank_C_matrix(vertices_list, faces_list)

num_boxes_x = 10; num_boxes_y = 1; num_boxes_z = 1
vertices_list, faces_list = generate_multi_box(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z)
@btime C = blank_C_matrix(vertices_list, faces_list)

## Section 7: More Functions 

In [ ]:
function create_B_app(vertices_list, direction, magnitude)
    N = length(vertices_list)
    B_app = zeros(3*N)
    
    for n in 1:N
        B_app[(n-1)*3+1:n*3] .= magnitude * direction
    end

    return B_app
end

function create_AA_chi(vertices_list, chi, C)
    N = length(vertices_list)
    A_chi = I(3*N) - chi / (1.25663706212e-6 * (1 + chi)) * C
    return A_chi
end

function find_M(A_chi, chi, B_app)
    M = A_chi \ (chi / (1.25663706212e-6 * (1 + chi)) * B_app)
    return M
end




In [ ]:
length_box = 1
width = 1
height = 1

num_boxes_x = 10
num_boxes_y = 1
num_boxes_z = 1

vertices_list, faces_list = generate_multi_box(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z)
C = build_C_matrix(vertices_list, faces_list)

direction_B_0 = [1, 0, 0]
magnitude = 1e-5
chi = 100

B_app = create_B_app(vertices_list, direction_B_0, magnitude)
A_chi = create_A_chi(vertices_list, chi, C)
M = find_M(A_chi, chi, B_app)



In [ ]:
length_box = 1
width = 1
height = 1


for i in 1:7
    num_boxes_x = i
    num_boxes_y = i
    num_boxes_z = i

    total_boxes = num_boxes_x * num_boxes_y * num_boxes_z
    print("Iteration $i - Total number of boxes: $total_boxes")
    @time begin
        vertices_list, faces_list = generate_multi_box(length_box, width, height, num_boxes_x, num_boxes_y, num_boxes_z)
        C = build_C_matrix(vertices_list, faces_list)
    end
    println()
    @time begin
        direction_B_0 = [1, 0, 0]
        magnitude = 1e-5
        chi = 100

        B_app = create_B_app(vertices_list, direction_B_0, magnitude)
        A_chi = create_A_chi(vertices_list, chi, C)
        M = find_M(A_chi, chi, B_app)
    end
    M
    println()
end